In [10]:
#trying to understand how to adjust τ based on initial drt data
using EISAnalysis

In [ ]:
function tuning(ω_exp,Z_exp;ppd=7,tol = 1e-04)
    # fit = compute_drt(ω_exp,Z_exp)
    τ_init = logrange(0.1/maximum(ω_exp),10/minimum(ω_exp),floor(Int,log10(100*maximum(ω_exp)/minimum(ω_exp)))*ppd)
    # τ_init = logrange(1/maximum(ω_exp),1/minimum(ω_exp),floor(Int,log10(maximum(ω_exp)/minimum(ω_exp)))*ppd)#testing
    ω_init = 1 ./τ_init
    n = length(ω_init) + 1
    dlnτ = log(τ_init[end]/τ_init[end-1])

    #cutting down data if it's too dense
    while length(ω_exp)>=length(ω_init)
        ω_exp = ω_exp[1:2:end]
        Z_exp = Z_exp[1:2:end]
    end

    Z_real,Z_imag = build_Z_matrices(ω_exp,ω_init)

    function drt_fit(ω,p)
        Z_drt = drt_Z(Z_real,Z_imag,p)
        return vcat(real(Z_drt),imag(Z_drt))
    end 
    fit_funct = drt_fit
    p0 = abs.(rand(n))
    fit = curve_fit(fit_funct, ω_exp, vcat(real(Z_exp),imag(Z_exp)), p0;lower = zeros(n),autodiff=:forwarddiff)
    p = fit.param
    γ_init = p[2:end]/dlnτ
    γ_max = maximum(γ_init)
    peaks = findmaxima(γ_init)
    peaks = peakheights(peaks,min = tol*γ_max)
    τ_pks,γ_pks = peaks.indices,peaks.heights
    Z_im = ω-> sum([γ_pks[i]*τ_pks[i]*10^ω/(1+τ_pks[i]^2*10^2ω) 
                    for i in eachindex(γ_pks)])

    #I'm trying to account for just local maxima
    γ_max0 = Z_im(1/τ_pks[1])
    γ_max1 = Z_im(1/τ_pks[end])
    Z_im_min = ω-> Z_im(ω) - tol*γ_max0*10 #relieving the tolerance slightly
    Z_im_max = ω-> Z_im(ω) - tol*γ_max1*10
    min = find_zeros(Z_im_min,-10,10)[1]
    max = find_zeros(Z_im_max,-10,10)[2]
    τ_min,τ_max = 10^-max,10^-min
    τ_tuned = logrange(τ_min,τ_max,floor(Int,log10(τ_max/τ_min))*ppd)
    return τ_tuned
end

tuning (generic function with 1 method)

In [21]:
ω_exp = [1,2,3]
maxa = 3

3

In [28]:
min(10.0^-maxa ,0.1/maximum(ω_exp)),max(10.0/minimum(ω_exp),10.0^-maxa)

(0.001, 10.0)

0.001